### Praktikum 6: File Handling Tambahan (Log) 

In [10]:
import pandas as pd
import folium
import datetime
from abc import ABC, abstractmethod

# --- Kelas Abstrak Lokasi ---
class Lokasi(ABC):
    def __init__(self, nama: str, lat: float, lon: float):
        self.nama = nama or "Tanpa Nama"
        try:
            self.lat = float(lat)
            self.lon = float(lon)
        except ValueError:
            self.lat = 0.0
            self.lon = 0.0

    def koordinat(self) -> tuple:
        return (self.lat, self.lon)

    @abstractmethod
    def popup_info(self) -> str:
        pass

    def __repr__(self) -> str:
        return f"{type(self).__name__}(nama='{self.nama}', lat={self.lat:.4f}, lon={self.lon:.4f})"

    def __str__(self) -> str:
        return f"{self.nama} ({type(self).__name__})"

# --- Subclass Lokasi ---
class TempatWisata(Lokasi):
    def __init__(self, nama, lat, lon, jenis, deskripsi):
        super().__init__(nama, lat, lon)
        self.jenis = jenis or "Umum"
        self.deskripsi = deskripsi or "Belum ada deskripsi."

    def popup_info(self):
        return f"<h4>{self.nama}</h4><i>{self.jenis}</i><br><br>{self.deskripsi}<br><br>Koordinat: ({self.lat:.4f}, {self.lon:.4f})"

class Kuliner(Lokasi):
    def __init__(self, nama, lat, lon, menu):
        super().__init__(nama, lat, lon)
        self.menu = menu or "Tidak tersedia."

    def popup_info(self):
        return f"<h4>{self.nama}</h4><i>Kuliner</i><br><br>Menu: {self.menu}<br><br>Koordinat: ({self.lat:.4f}, {self.lon:.4f})"

class TempatIbadah(Lokasi):
    def __init__(self, nama, lat, lon, agama="Umum", deskripsi="Tempat ibadah"):
        super().__init__(nama, lat, lon)
        self.agama = agama
        self.deskripsi = deskripsi

    def popup_info(self):
        return f"<h4>{self.nama}</h4><i>{self.agama}</i><br><br>{self.deskripsi}<br><br>Koordinat: ({self.lat:.4f}, {self.lon:.4f})"

class Museum(Lokasi):
    def __init__(self, nama, lat, lon, deskripsi="Museum umum."):
        super().__init__(nama, lat, lon)
        self.deskripsi = deskripsi

    def popup_info(self):
        return f"<h4>{self.nama}</h4><i>Museum</i><br><br>{self.deskripsi}<br><br>Koordinat: ({self.lat:.4f}, {self.lon:.4f})"

class Kantor(Lokasi):
    def __init__(self, nama, lat, lon, deskripsi="Gedung pemerintahan."):
        super().__init__(nama, lat, lon)
        self.deskripsi = deskripsi

    def popup_info(self):
        return f"<h4>{self.nama}</h4><i>Kantor Pemerintahan</i><br><br>{self.deskripsi}<br><br>Koordinat: ({self.lat:.4f}, {self.lon:.4f})"

class Taman(Lokasi):
    def __init__(self, nama, lat, lon, deskripsi="Taman kota terbuka."):
        super().__init__(nama, lat, lon)
        self.deskripsi = deskripsi

    def popup_info(self):
        return f"<h4>{self.nama}</h4><i>Taman Kota</i><br><br>{self.deskripsi}<br><br>Koordinat: ({self.lat:.4f}, {self.lon:.4f})"

# --- Fungsi untuk membaca CSV ---
def muat_data_csv(nama_file: str) -> pd.DataFrame | None:
    try:
        return pd.read_csv(nama_file)
    except FileNotFoundError:
        print(f"[Gagal] File {nama_file} tidak ditemukan.")
    except Exception as e:
        print(f"[Kesalahan] {type(e).__name__}: {e}")
    return None

# --- Parsing Lokasi dari DataFrame ---
def parsing_lokasi(df: pd.DataFrame) -> list:
    daftar_lokasi = []
    if df is None or df.empty:
        return daftar_lokasi

    for _, row in df.iterrows():
        nama = row.get('Nama')
        lat = row.get('Latitude')
        lon = row.get('Longitude')
        tipe = row.get('Tipe', 'Lain')
        deskripsi = row.get('Deskripsi', '')

        if not all([nama, lat, lon]):
            continue

        objek = None
        try:
            match tipe:
                case t if 'Wisata' in t or t == 'Landmark':
                    objek = TempatWisata(nama, lat, lon, tipe, deskripsi)
                case 'Kuliner':
                    objek = Kuliner(nama, lat, lon, deskripsi)
                case t if 'Ibadah' in t:
                    objek = TempatIbadah(nama, lat, lon, "Umum", deskripsi)
                case 'Kantor Pemerintahan':
                    objek = Kantor(nama, lat, lon, deskripsi)
                case 'Museum':
                    objek = Museum(nama, lat, lon, deskripsi)
                case 'Taman Kota':
                    objek = Taman(nama, lat, lon, deskripsi)
        except Exception as e:
            print(f"Error parsing '{nama}': {e}")

        if objek:
            daftar_lokasi.append(objek)

    return daftar_lokasi



# --- Baca konfigurasi dari file config_peta.txt ---
def baca_konfigurasi(file_config="config_peta.txt"):
    default_lokasi = (-6.9929, 110.4200)
    default_zoom = 13

    try:
        with open(file_config, 'r', encoding='utf-8') as f:
            baris = [line.strip() for line in f.readlines()]
            if len(baris) < 3:
                raise IndexError("Baris kurang dari 3.")
            lat = float(baris[0])
            lon = float(baris[1])
            zoom = int(baris[2])
            return (lat, lon), zoom
    except (FileNotFoundError, ValueError, IndexError) as e:
        print(f"[Konfigurasi] Gagal membaca file konfigurasi: {e}")
        return default_lokasi, default_zoom

# --- Buat peta dengan marker sesuai tipe lokasi ---
def buat_peta(daftar_objek: list, file_html="peta.html", lokasi_awal=None, zoom=12):
    if not daftar_objek:
        print("Tidak ada data untuk dipetakan.")
        return

    if not lokasi_awal:
        lokasi_awal = (-6.9929, 110.4200)

    peta = folium.Map(location=lokasi_awal, zoom_start=zoom)

    for obj in daftar_objek:
        latlon = obj.koordinat()
        if latlon == (0.0, 0.0):
            continue

        if isinstance(obj, TempatWisata): warna, ikon = 'blue', 'camera'
        elif isinstance(obj, Kuliner): warna, ikon = 'red', 'cutlery'
        elif isinstance(obj, TempatIbadah): warna, ikon = 'purple', 'place-of-worship'
        elif isinstance(obj, Museum): warna, ikon = 'darkblue', 'university'
        elif isinstance(obj, Kantor): warna, ikon = 'gray', 'building'
        elif isinstance(obj, Taman): warna, ikon = 'green', 'tree'
        else: warna, ikon = 'cadetblue', 'info-sign'

        folium.Marker(
            location=latlon,
            popup=folium.Popup(obj.popup_info(), max_width=250),
            tooltip=obj.nama,
            icon=folium.Icon(color=warna, icon=ikon, prefix='fa')
        ).add_to(peta)

    try:
        peta.save(file_html)
        print(f"Peta berhasil disimpan ke {file_html}")
    except Exception as e:
        print(f"[Gagal] Menyimpan peta: {e}")

# --- MAIN ---
if __name__ == "__main__":
    FILE_CSV = "lokasi_semarang.csv"
    FILE_HTML = "PETA_INTERAKTIF_SEMARANG_BARU.html"
    FILE_CONFIG = "config_peta.txt"

    data = muat_data_csv(FILE_CSV)
    objek_lokasi = parsing_lokasi(data)
    lokasi_awal, zoom_awal = baca_konfigurasi(FILE_CONFIG)
    buat_peta(objek_lokasi, FILE_HTML, lokasi_awal, zoom_awal)
    print("-----Selesai-----")


Peta berhasil disimpan ke PETA_INTERAKTIF_SEMARANG_BARU.html
-----Selesai-----
